In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# main constants

# reference distance (meters)
d0 = 1 
# constant for particular propagation enviroment
k = 1e-4
# path loss exponent
n = 4
# total available bandwidth (Hz)
b_t = 100 * 1e6
# constant for noise power (W/Hz)
k_0 = 1e-20
# Side of total area (meters)
area_side = 1e3

In [3]:
def lin2db(x):
    return 10.0*np.log10(x)

def db2lin(x):
    return 10.0**(x/10.0)

def lin2dbm(x):
    return 10.0*np.log10(x)+30.0

def dbm2lin(x):
    return 10.0**(x/10.0 - 3.0)

In [4]:
def random_position():
    
    x_position = np.random.rand()*area_side
    y_position = np.random.rand()*area_side
    
    return [x_position, y_position]

def distance(UE_position, AP_position):

    return np.linalg.norm(np.array(UE_position)
                          - np.array(AP_position))

def AP_position(AP_quantity):
    
    # Generates a list of coordenates according to the number of APs
    # in the simulation. It distributes the APs evenly.

    # Checks if the number of APs is a perfect square.
    if np.sqrt(AP_quantity).is_integer(): 

        AP_quantity_side = int(np.sqrt(AP_quantity))
    
        cell_side = area_side // AP_quantity_side
    
        # Creates coordenates using meshgrid.
        x, y = np.meshgrid(np.arange(0.5 * cell_side, 
                                     area_side, 
                                     cell_side),
                           np.arange(0.5 * cell_side, 
                                     area_side, 
                                     cell_side))
        
        APs_positions = np.column_stack((x.ravel(), y.ravel()))
    else:
        print("AP quantity must be a perfect square")

    return APs_positions

In [5]:
# Functions to calculate equations given by the base material

def calculate_received_power(transmitted_power, shadowing, distance):
    return transmitted_power * shadowing * (k / (distance**n))
    
def calculate_bandwidth_channel(total_bandwidth, channels_quantity):
    return total_bandwidth / channels_quantity

def calculate_noise_power(total_bandwidth, channels_quantity):
    return k_0 * calculate_bandwidth_channel(total_bandwidth, channels_quantity)

def calculate_SNR(user_power, noise_power):
    return user_power / noise_power

def calculate_SINR(user_power, interference_power, noise_power):
    return user_power / (interference_power + noise_power)

In [6]:
def eCDF(data):
    x = np.sort(data)
    y = np.arange(0, len(data)) / len(data)

    return x, y  

In [7]:
# Models the UEs
class UserEquipment:
    
    def __init__(self, transmitted_power_ = 1): # UE transmission power is 1 W by standard
        
        # Distance is sorted randomly in a 1000x1000m area 
        self.position = random_position()
        self.distance2AP = 0
        
        self.transmitted_power = transmitted_power_
        self.received_power = 0

In [8]:
# Models the APs
class AccessPoint:

    def __init__(self, channels_quantity_, position_ = [0, 0]):

        self.position = position_

        # Adds keys with a empty list for each channel in the system
        self.channels = dict()
        for m in range(channels_quantity_):
            self.channels[m + 1] = []

In [9]:
# CLass that plots the APs and UEs
class LinkSystem:
    
    def __init__(self, AP_quantity_, UE_quantity_, channels_quantity_):

        self.AP_quantity = AP_quantity_
        self.AP_list = []
        
        self.UE_quantity = UE_quantity_
        self.UE_list = []
        
        self.channels_quantity = channels_quantity_

        self.noise_power = calculate_noise_power(b_t, self.channels_quantity)

        
        ''' Matrix of shadowing of each link, in which the rows are related to UEs
        and columns are related to APs '''
        self.shadowing_matrix = np.zeros((self.UE_quantity, 
                                     self.AP_quantity))
        
        for UE in range(self.UE_quantity):
            for AP in range(self.AP_quantity):
                self.shadowing_matrix[UE][AP] = np.random.lognormal(sigma=2.0)  

        
        ''' Add APs in system '''
        positions = AP_position(self.AP_quantity)
        index = 0
        
        for AP in range(self.AP_quantity):
            
            self.AP_list.append(AccessPoint(self.channels_quantity, positions[index]))
            index += 1

        # Add UEs in system
        for j in range(UE_quantity_):
            
            self.UE_list.append(UserEquipment())

        
        ''' Matrix of shadowing of each link, in which the rows are related to UEs
        and columns are related to APs '''
        self.gain_matrix = np.zeros((self.UE_quantity,
                                self.AP_quantity))
        
        for UE in self.UE_list:
            for AP in self.AP_list:
                gain = ((self.shadowing_matrix[self.UE_list.index(UE)][self.AP_list.index(AP)]
                        * k) / distance(UE.position, AP.position))
                
                self.gain_matrix[self.UE_list.index(UE)][self.AP_list.index(AP)] = gain
                

        ''' Stores the linked AP from each UE according to channel
        and allocate them in this AP channels '''
        for UE in range(self.UE_quantity):

            linkedAP = 0
            min_gain = float('-inf')
            
            for AP in range(self.AP_quantity):
                if self.gain_matrix[UE][AP] > min_gain:
                    linkedAP = self.AP_list[AP]
                    min_gain = self.gain_matrix[UE][AP]
                
            
            ''' Uses the AP with better channel, that it stored in the loop above.
            Stores the better channel AP from each UE and allocate them in 
            this AP channels, calculating its distance to linked AP. '''

            if self.channels_quantity > 1:
                linkedAP.channels[np.random.randint(1, self.channels_quantity)].append(self.UE_list[UE])
            else:
                linkedAP.channels[1].append(self.UE_list[UE])
                
            self.UE_list[UE].distance2AP = distance(self.UE_list[UE].position, 
                                                    linkedAP.position)


        
        self.SNR_list = []
        self.SINR_list = []
        self.spectral_efficiency_list = []
            
        ''' It calculates each UE KPI for each AP channel '''
        for AP in self.AP_list:
                                
            for channel in AP.channels:
                
                # Power sum of all UEs in the channel
                power_sum = 0
                
                # Calculates the received power of each UE
                for UE in AP.channels[channel]:           

                    # Calculates the received power of each UE
                    UE.received_power = calculate_received_power(UE.transmitted_power, 
                                                                 self.shadowing_matrix[self.UE_list.index(UE)][self.AP_list.index(AP)],
                                                                 UE.distance2AP)
                    
                    power_sum += UE.received_power
                        
                    # Calculating SNR
                    for UE in AP.channels[channel]:
                        self.SNR_list.append(lin2db(calculate_SNR(UE.received_power, 
                                                                  self.noise_power)))

                    # Calculating SINR
                    for UE in AP.channels[channel]:
                        UE_used = UE.received_power
                        interfering_sum = power_sum - UE_used
                        
                        self.SINR_list.append(lin2db(UE_used 
                                                     / (interfering_sum 
                                                     + self.noise_power)))

                        # self.SINR_list[user_] = lin2db(calculate_SINR(UE_used, interfering_sum, noise_p))
                        # The function is not working for some reason I will take care later
            
            # Calculating Spectral Efficiency
            for user_SINR_ in range(len(self.SINR_list)):
                bandwidth_ = calculate_bandwidth_channel(b_t, self.channels_quantity)
                self.spectral_efficiency_list.append(np.log2(1 
                                                     + db2lin(self.SINR_list[user_SINR_])))


        
    def centralized_power_control(self):
        
        matrix_F = np.zeros((self.UE_quantity, self.AP_quantity))

        for i in range(self.UE_quantity):
            for j in range(self.AP_quantity):

                if i == j:
                    matrix_F[i][j] = 0
                else:
                    matrix_F[i][j] = (self.gain_matrix[j][i]
                                      / self.gain_matrix[i][i])

        print(matrix_F)

In [10]:
# Simulation parameters

iteracoes = 1 # The quantity of simulations
# It only works with i = j ?????
users_quantity = 5
AP_quantity = 4
channels_quantity = 1

In [11]:
total_SNR = []
total_SINR = []
total_spectral_efficiency = []

for inter in range(iteracoes):
    
    # Creates many Users at once
    system = LinkSystem(AP_quantity, 
                        users_quantity, 
                        channels_quantity)

    system.centralized_power_control()
    
    # The SNR, SINR and spectral efficiency of users    
    total_SNR = np.concatenate((total_SNR, 
                                system.SNR_list))
    
    total_SINR = np.concatenate((total_SINR, 
                                 system.SINR_list))
    
    total_spectral_efficiency = np.concatenate((total_spectral_efficiency, 
                                                system.spectral_efficiency_list))

/tmp/ipykernel_18288/630489032.py:2: RuntimeWarning: divide by zero encountered in log10
  return 10.0*np.log10(x)


IndexError: index 4 is out of bounds for axis 0 with size 4

In [ ]:
# spectral_efficiencyCDF = eCDF(total_spectral_efficiency)
# SINR_CDF = eCDF(total_SINR)
# SNR_CDF = eCDF(total_SNR)

# plt.title('Eficiência Espectral')
# plt.xlabel('Eficiência Espectral [bps/Hz, linear]')
# plt.ylabel('CDF')

# plt.ylim(0.0, 1.0)
# plt.plot(spectral_efficiencyCDF[0], spectral_efficiencyCDF[1])
# plt.grid()